python:3.9.13

tensorflow:2.18.0

torch:2.6.0

----

每局只取1个样本：慢  

缓慢降温增加探索性：效果不好。奖励只在最后一步才出现，探索性破坏奖励信号？  

更小卷积核：测试效果很差  

q-learning：测试效果很差  

sarsa：测试效果很差  

补充人工信息：效果提升

增加层数：加了两层，大部分时候表现优于aug，但是有交叉。

降低 learning rate: 从 1e-3 降低到 1e-4, 表现提升。

----

目前aug(base4基础上，输入数据增加了人工信息) 略优于 base4(补上了base2遗漏的batch normalization 和 dropout)

bigger(增加了两层) 略优于 aug

出牌信息里去掉带牌？可能意义不大

为什么训练过程中表现有起伏呢？降低 learning rate？增加每个轮次的局数？DQN？

如果DQN的话，要怎么做呢？DQN使用 experience buffer 是为了降低样本的相关性。在斗地主中，如果我一次收集很多局的样本，然后把样本打乱，喂给模型，这样就达到了降低样本相关性的目的吧。在这种情况下，policy network 本身就不是每一步都更新的。还有必要区分 policy network 和 target network 吗？

按照DQN的逻辑，我是不是可以把同一批样本反复随机分割成batch喂给模型？可以加快训练速度吗？

之前 q-learning 的效果太差了，完全没信心

----

trainer输出的loss意义有限。如果发牌高度偏向于地主或农民，那么预测就更准，loss就会更小。一轮只有8局，发牌的影响很大。

----

感觉 DouZero 所谓 Deep Monte-Carlo 跟我的逻辑是一样的。打牌，生成一整局游戏的样本，用最终的输赢作为预测目标，训练状态动作价值模型。多线程不熟悉代码不怎么看得懂，似乎一次生成50局游戏，然后 batch size 32

----
因为电脑是8核，之前都是每8局游戏更新模型。地主农民分三个模型之后，表现下降。可能是因为每个模型的样本数量只有之前的1/3。相应地改成每24局游戏更新一次，似乎表现提升，但是训练速度当然慢了很多。

一个问题，把收集到的样本打乱，32个样本一个batch，不满32个的也当成一个batch，是不是有问题？不满32个的应该怎么处理。假如某组只有1个样本，是不是不太好？

loss 修正: loss_factor = (len(xs_batch) / batch_size) ** 0.5  似无优势，可能训练过程略微平稳一些

----

怎么把douzero模型下载并跑起来……

----

tensorflow 模型 1696257 参数，运行速度 13ms，torch 模型 1457665 参数，运行速度 4ms。另外的我的模型需要的训练数据，组装速度也较慢，主要是组装额外数据(aug加入)消耗了额外的时间。

----

torch 比 tensorflow 快很多。发现用 torch 时，如果 batch size 为 1 会报错。所以 bn 是在一个 batch 内做 normalization？感觉不对劲。  
BatchNorm2D: batch 内所有样本同一个 channel 所有数据做 normalization, scaled(x[:, idx_channel, :, :])  
LayerNorm: batch 内每个样本单独做 normalization, scaled(x[idx_sample, :, :, :])  
算了，改用 LayerNorm 吧  
发现 LayerNorm 更平稳。为什么？ 直觉上似乎 batch normalization 更合理？因为有少数 batch size 不是固定的32？ batch size 太小？

----

似乎遇到内存泄露，跑了一晚上，报错停止了。  
使用 with mp.Pool() as pool, 内存稳定，但速度慢很多  
使用 del, 无效  
使用 del 和 gc.collect(), 无效，速度慢  
将就一下，仍旧使用 pool = mp.Poo() 以及每200轮关闭和重建创建 pool

----

公司个人办公电脑效率测试：trainer_torch_ln, 两分钟
cpu| rounds
----|----
1| 59
2| 79
3| 90
4| 94
5| 98
6| 102

公司linux服务器效率测试：trainer_torch_ln, 两分钟
cpu|rounds
----|----
1| 50
2| 58
3| 38
4| 13
6| 4

同样的代码，在我的公司电脑上运行的速度是在公司服务器/阿里云服务器上的20多倍。为什么？linux的问题？从资源占用来看，服务器上的cpu也是一直在工作的。  
找到原因了。模型计算时默认使用多核，各进程的模型互相抢CPU。不知道为什么这个问题在个人PC上没那么严重，windows和linux的差异？  
torch.set_num_threads 设置使用模型使用的线程数。在linux服务器上设置12进程，子进程 torch.set_num_threads(1) 之后，两分钟跑了80轮  
本地PC采用8进程和torch.set_num_threads(1)之后，运行速度似有些微提升。

-----
使用 python multiprocessing 多进程遇到的问题：  
内存占用不断增加：使用 with mp.Pool() as pool 可以避免这个问题，但是会降低程序运行速度。改成在循环中每隔若干轮关闭并重建pool。  
使用 Process(target, args)，不一定使用多核。不知道如何指定使用几个进程。即使使用多核，使用队列传递数据，程序运行速度相比 pool.map 也不见优势。  
使用多进程，运行速度没有线性增加。  
使用6核，设定每个 selfplay 进行一局游戏，执行24次，或者设定每个selfplay 进行4局游戏，执行6次，运行效率没区别。但把总执行次数改成48局，前一种会稍慢。说明复制参数或收集返回结果确实有计算成本，只是在本电脑本问题上不是很显著。  

----
Dropout: 同样的输入，在 train 模式下，输出会不一样。但是如果输出的方差很大，这会不会是个问题？  
刚才是用随机生成的数据测试。重新用实际的牌局数据测试，方差不大。

----
不合理出牌：  
bot_torch_ln: 316300  

```
0.99337137 6 A A A
0.9860233 6 6 A A A
0.985545 3 A A A
0.9849531 3
0.9831738 A A A
21 :  地主 : 6 A A A | 3 6 大王
22 : 农民1 :  | 4 5 7 7 J Q Q K K 2
23 : 农民2 :  | 3 3 4 4 4 5 5 10 2 2
```

----

每100轮(24局1轮)存1次模型，与douzero比较，地主农民各1000局，统计45万轮之后各模型的胜利局数。分开看做地主和做农民时的胜利局数，统计的方差和基于胜率估算的方差相差不大。但是把做地主和做农民胜利的局数加起来看，统计的方差和基于胜率估算的方差相差很大。地主胜率和农民胜率相关系数 -0.6。这种情况给人什么感觉呢？好像模型实力已经不怎么提升了，只是在左右互搏的过程中限入了某种循环状态。类似 地主策略 A > 农民策略 A > 地主策略B > 农民策略 B > 地主策略 A

使用最近200轮的模型池生成样本能避免这种情况吗？目前从30万轮起，跑了7万轮，地主胜率和农民胜率相关系数同样 -0.6。继续跑看看。

上面的猜测好像不对。测试的时候，每组发牌打两局，一局当地主，一局当农民。如果测试某一轮的模型时，发的牌整体偏向地主，地主胜率就高，农民胜率就低

----

从前200个模型中随机抽取模型对战生成样本，避免可能的策略过拟合：看不出来效果。

----

根据模型打牌，生成一个局面信息的序列。然后在序列的每一步，选择其他（最多4手）评分高的出牌选项，接下来再按模型打牌，生成序列。  
如此得到一个局面信息的树，每一层有若干个子结点，对应当前局面下的不同出牌，但只有一个子结点（最高胜率的出牌）有分支，其他的都是一条线直到叶结点。  
即先按模型生成主干，然后在主干上的每一步分出分支，每个分支不再有新的分支，而是按模型一路走到底。  
希望具有的优点：1, 一手牌提供更多的训练数据。2, 同一个局面下的不同出牌的奖励对比能提供更有效的训练信号。  
还在训练中……  
试验几种设置：一轮6局，batch size=32 | 一轮24局，batch size=32 | 一轮24局，batch size=128  
一轮24局，batch size=128 较好  
训练似乎较慢。每局游戏生成的样本数量大大增加（50多倍），每一轮的耗时也大大增加。虽然样本多，但对应的初始发牌少，训练速度慢可能与此有关。继续训练希望表现能更好。

表现似乎有所提升，对douzero胜率大概提升了0.5%

----
不作弊的模型  
128局一轮优于24局一轮  
增加自己手牌的顺子、连对、飞机信息，表现较大提升  
使用 lstm_out 和 h_n 作为下一步的输入，表现无差异

# 加载包

In [1]:
import numpy as np
import torch
import rules
import arena as arn
import bot_torch_ln
CARDS = rules.CARDS
ARENA = arn.ARENA
bot_rd = arn.bot_rd
bot_me = arn.bot_me


In [2]:
import bot_douzero
import importlib
importlib.reload(bot_douzero)

<module 'bot_douzero' from 'd:\\py\\ddz_2025_2\\bot_douzero.py'>

In [3]:
def ft(N, bot1_f, bot2_f):
    s1, s2 = 0, 0
    for i in range(N):
        cards = rules.CARDS.copy()
        np.random.shuffle(cards)
        
        arena = ARENA(verbos=0, cards=cards.copy())
        arena.registerbot([bot1_f(), bot2_f(), bot2_f()])
        arena.wholegame()
        s1 += arena.winner == 0

        arena = ARENA(verbos=0, cards=cards.copy())
        arena.registerbot([bot2_f(), bot1_f(), bot1_f()])
        arena.wholegame()
        s2 += arena.winner == 0

    return s1, s2

# models & bots

In [ ]:
# 7489: 2143, 1644 | 5499/10000
# 7712: 2126, 1682 | 5440/10000
#12762: 2127, 1509 | 5618/10000
#12782: 2106, 1478 | 5628/10000
#13339: 2163, 1494 | 5669/10000
#14315: 2188, 1509 | 5679/10000  vs model_torch_ln/cp701000: 1677, 1630 / 5000
#14384: 2026, 1439 | 5587/10000
#14427: 2036, 1507 | 5529/10000
#14671: 2088, 1481 | 5607/10000
#15783: 2093, 1436 | 5657/10000
#77795: 2260, 1368 | 5892/10000
#85570: 
cp_tree = torch.load("model_tree2/cp77795.pt")
models_tree = [bot_torch_ln.Model() for _ in range(3)]
for pos in range(3):
    # models_tree[pos].eval()
    models_tree[pos].load_state_dict(cp_tree["models_state_dict"][pos])

bot_tree_f = lambda verbos=0: bot_torch_ln.BOT(models_tree, verbos=verbos)


In [19]:
bot_douzero_f = lambda verbos=0: bot_douzero.BOT(verbos)

In [25]:
N = 5000
ft(N, bot_tree_f, bot_douzero_f)

(2260, 1368)

# 示例

In [ ]:
arena = arn.ARENA(1, True)
arena.registerbot([bot_tree_f(1), bot_tree_f(1), bot_tree_f(1)])
arena.wholegame()

0.001924147 7
0.0018158875 6
0.0017111086 5
0.0014695198 J
0.0012324355 4 4
 0 :  地主 : 7 | 3 3 4 4 5 5 6 7 8 9 J Q Q K K K 2 2 小王
0.9963905 Q
0.99562293 J
0.9937418 A
0.9921051 
0.9881852 2
 1 : 农民1 : Q | 4 5 6 7 7 8 9 10 10 10 10 J A A 2 2
0.9973011 
0.9968774 K
0.995038 A
0.98727274 大王
 2 : 农民2 :  | 3 3 4 5 6 6 8 8 9 9 J J Q K A A 大王
0.0011281408 小王
0.00096688286 
0.0009559222 K
0.00063989934 2
 3 :  地主 : 小王 | 3 3 4 4 5 5 6 7 8 9 J Q Q K K K 2 2
0.9965637 
0.9904817 10 10 10 10
 4 : 农民1 :  | 4 5 6 7 7 8 9 10 10 10 10 J A A 2 2
0.99784726 
0.99650764 大王
 5 : 农民2 :  | 3 3 4 5 6 6 8 8 9 9 J J Q K A A 大王
0.00093064236 5
0.00071315997 J
0.0006391631 4
0.00062739226 3 3
0.00059046503 4 4
 6 :  地主 : 5 | 3 3 4 4 5 6 7 8 9 J Q Q K K K 2 2
0.9993014 7
0.9992085 J
0.99870485 A
0.9983054 2
0.9980009 
 7 : 农民1 : 7 | 4 5 6 7 8 9 10 10 10 10 J A A 2 2
0.99920106 K
0.9991605 Q
0.9989387 J
0.99893594 9
0.9989352 8
 8 : 农民2 : K | 3 3 4 5 6 6 8 8 9 9 J J Q A A 大王
0.000680071 
0.00022084691 2
 9 :  地主 :

# 同样发牌与douzero交换地主农民

In [22]:
while True:
    cards = CARDS.copy()

    np.random.shuffle(cards)
    arena = ARENA(cards=cards.copy())
    arena.registerbot([bot_tree_f(), bot_douzero_f(), bot_douzero_f()])
    arena.wholegame()
    w1 = arena.winner
    
    arena = ARENA(cards=cards.copy())
    arena.registerbot([bot_douzero_f(), bot_tree_f(), bot_tree_f()])
    arena.wholegame()
    w2 = arena.winner
    
    if (w1==0) != (w2==0):
        arena = ARENA(cards=cards)
        break

In [23]:
arena = ARENA(1, cards=cards.copy())
arena.registerbot([bot_tree_f(1), bot_douzero_f(1), bot_douzero_f(1)])
arena.wholegame()

0.94567627 5 6 7 8 9 10
0.77947974 6
0.7211424 8
0.7210897 9
0.55939525 2
 0 :  地主 : 5 6 7 8 9 10 | 3 3 6 8 9 Q Q Q A A A A 2 大王
 1 : 农民1 :  | 3 3 4 5 5 5 6 6 7 7 8 9 9 J Q K 2
 2 : 农民2 :  | 4 4 4 7 8 10 10 10 J J J K K K 2 2 小王
0.83883345 8 A A A A 2
0.78527385 6
0.78090936 8
0.77899206 8 9 A A A A
0.7784549 9
 3 :  地主 : 8 A A A A 2 | 3 3 6 9 Q Q Q 大王
 4 : 农民1 :  | 3 3 4 5 5 5 6 6 7 7 8 9 9 J Q K 2
 5 : 农民2 :  | 4 4 4 7 8 10 10 10 J J J K K K 2 2 小王
0.40961647 9
0.19284216 6
0.06481005 9 Q Q Q
0.062139694 3 3
0.05925912 6 Q Q Q
 6 :  地主 : 9 | 3 3 6 Q Q Q 大王
-1.0203382 
-1.0227846 K
-1.023124 J
-1.0242015 Q
-1.0544302 2
 7 : 农民1 :  | 3 3 4 5 5 5 6 6 7 7 8 9 9 J Q K 2
-1.0130631 
-1.0147163 2
-1.0149857 小王
-1.0158393 J
-1.0181823 K
 8 : 农民2 :  | 4 4 4 7 8 10 10 10 J J J K K K 2 2 小王
0.99724543 6
0.20492183 大王
0.14095542 3 3 Q Q Q
0.125296 3 3
0.11618759 3
 9 :  地主 : 6 | 3 3 Q Q Q 大王
-1.0468855 J
-1.0527043 Q
-1.0528686 
-1.0535328 K
-1.0594697 8
10 : 农民1 : J | 3 3 4 5 5 5 6 6 7 7 8 9 9 

In [24]:
arena = ARENA(1, cards=cards.copy())
arena.registerbot([bot_douzero_f(1), bot_tree_f(1), bot_tree_f(1)])
arena.wholegame()

0.32266718 3 3
0.31932092 6
0.30529118 9
0.25549766 8
0.22369164 3 3 Q Q Q
 0 :  地主 : 3 3 | 5 6 6 7 8 8 9 9 10 Q Q Q A A A A 2 大王
0.44336993 
0.36075586 6 6
0.35811073 5 5
0.350878 9 9
0.33837384 7 7
 1 : 农民1 :  | 3 3 4 5 5 5 6 6 7 7 8 9 9 J Q K 2
0.43711764 2 2
0.3882454 4 4
0.12157291 
0.09630108 10 10
0.09230602 J J
 2 : 农民2 : 2 2 | 4 4 4 7 8 10 10 10 J J J K K K 小王
0.042548016 
-0.37764123 A A A A
 3 :  地主 :  | 5 6 6 7 8 8 9 9 10 Q Q Q A A A A 2 大王
0.3565604 
 4 : 农民1 :  | 3 3 4 5 5 5 6 6 7 7 8 9 9 J Q K 2
0.43297732 7 8 10 10 10 J J J
0.2745555 4 4 4 8
0.26434076 4 4 4
0.26092714 4 4 4 7
0.25114244 4 4 4 小王
 5 : 农民2 : 7 8 10 10 10 J J J | 4 4 4 K K K 小王
-0.3272178 A A A A
-0.39993426 
 6 :  地主 : A A A A | 5 6 6 7 8 8 9 9 10 Q Q Q 2 大王
0.72261107 
 7 : 农民1 :  | 3 3 4 5 5 5 6 6 7 7 8 9 9 J Q K 2
0.6098151 
 8 : 农民2 :  | 4 4 4 K K K 小王
-0.4190579 8
-0.44692945 9
-0.5303711 6
-0.55426025 6 6
-0.57029605 7
 9 :  地主 : 8 | 5 6 6 7 8 9 9 10 Q Q Q 2 大王
0.7670903 
0.7481717 9
0.74356174 J
0